In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [2]:
org_df_policy = pd.read_csv ('usa-covid-policy/data/OxCGRT_US_latest.csv')
org_jhu_deaths_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
org_jhu_cases_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

/home/nick/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Oxford Policy Data

In [3]:
org_df_policy

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,United States,USA,NaN,NaN,NAT_GOV,20200101,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
1,United States,USA,NaN,NaN,NAT_GOV,20200101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,USA,NaN,NaN,NAT_GOV,20200102,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,United States,USA,NaN,NaN,NAT_GOV,20200102,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,USA,NaN,NaN,NAT_GOV,20200103,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16249,United States,USA,Wyoming,US_WY,STATE_ALL,20201023,NaN,NaN,NaN,NaN,...,NaN,40.74,NaN,49.52,NaN,45.24,NaN,48.61,NaN,25.0
16250,United States,USA,Wyoming,US_WY,STATE_ALL,20201024,NaN,NaN,NaN,NaN,...,NaN,40.74,NaN,49.52,NaN,45.24,NaN,48.61,NaN,25.0
16251,United States,USA,Wyoming,US_WY,STATE_ALL,20201025,NaN,NaN,NaN,NaN,...,NaN,40.74,NaN,49.52,NaN,45.24,NaN,48.61,NaN,25.0
16252,United States,USA,Wyoming,US_WY,STATE_ALL,20201026,NaN,NaN,NaN,NaN,...,NaN,40.74,NaN,49.52,NaN,45.24,NaN,48.61,NaN,25.0


In [4]:
df_policy = org_df_policy[["Date", "RegionCode", "RegionName", "CountryCode", "Jurisdiction", "StringencyIndexForDisplay",
                           "GovernmentResponseIndexForDisplay", "ContainmentHealthIndexForDisplay",
                           "EconomicSupportIndexForDisplay"]]

In [5]:
df_policy.Jurisdiction.unique()
# Removing rows where the Jurisdiction is NAT_GOV since I only want to look at the states
nat_gov_jur = df_policy[df_policy["Jurisdiction"] == "NAT_GOV"].index
df_policy.drop(nat_gov_jur, inplace=True)

/home/nick/.local/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
# Normalize the Indicies Columns using the max for each col.
indices = df_policy.filter(regex="Index")
for col in indices.columns:
    df_policy[col] /= 100

# Each index is composed of a series of individual policy response indicators. For each
# indicator, we create a score by taking the ordinal value and adding an extra half-point
# if the policy is general rather than targeted, if applicable. We then rescale each of
# these by their maximum value to create a score between 0 and 100, with a missing
# value contributing 0.5 These scores are then averaged to get the composite indices.

<ipython-input-6-0220f3b0c3da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_policy[col] /= 100


In [7]:
df_policy = df_policy.reset_index().drop("index", axis=1)

In [8]:
df_policy = df_policy[df_policy['Date'] >= 20200122]


In [9]:
df_policy['Date'] = pd.to_datetime(df_policy['Date'], format='%Y%m%d')

In [10]:
states_in_policy = set(df_policy['RegionName'])

In [11]:
df_policy = df_policy.drop(['RegionCode', 'Jurisdiction'], axis=1)

In [12]:
# This cell replaces all Nan Values where the previous value == the next valid value 
# for the specified column - the Nan is replaced with the prev value
df_policy.loc[df_policy['StringencyIndexForDisplay'].ffill() == 
              df_policy['StringencyIndexForDisplay'].bfill(),
              'StringencyIndexForDisplay'] = df_policy['StringencyIndexForDisplay'].ffill()

df_policy.loc[df_policy['GovernmentResponseIndexForDisplay'].ffill() == 
              df_policy['GovernmentResponseIndexForDisplay'].bfill(),
              'GovernmentResponseIndexForDisplay'] = df_policy['GovernmentResponseIndexForDisplay'].ffill()

df_policy.loc[df_policy['ContainmentHealthIndexForDisplay'].ffill() == 
              df_policy['ContainmentHealthIndexForDisplay'].bfill(),
              'ContainmentHealthIndexForDisplay'] = df_policy['ContainmentHealthIndexForDisplay'].ffill()

df_policy.loc[df_policy['EconomicSupportIndexForDisplay'].ffill() == 
              df_policy['EconomicSupportIndexForDisplay'].bfill(),
              'EconomicSupportIndexForDisplay'] = df_policy['EconomicSupportIndexForDisplay'].ffill()

# Old code
# The following line are checking the EconomicSupportIndexForDisplay data for Idaho as
# There seems to be a NaN at 9/1 that is a one off - the surrounding numbers are zero so I will make it 0.
# y = df_policy.loc[(df_policy['RegionName']=='Idaho') & (df_policy['Date'] >= pd.Timestamp(2020,8,30))]

# The below line changes the value - for Idaho EconomicSupportIndexForDisplay on 9/1/2020 to zero
# df_policy.loc[(df_policy['RegionName']=='Idaho') &
#               (df_policy['Date'] == pd.Timestamp(2020,9,1)),'EconomicSupportIndexForDisplay'] = 0.0

In [13]:
# Checking there are no nulls in df_policy
null_policy = df_policy[df_policy.isnull().any(axis=1)]
null_policy 

,Date,RegionName,CountryCode,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay


# JHU Data: Covid-19 Deaths

In [14]:
df_deaths_jhu = org_jhu_deaths_df.groupby('Province_State').sum()
df_deaths_jhu = df_deaths_jhu.reset_index()

In [15]:
df_deaths_jhu['Province_State'].replace({"District of Columbia": "Washington DC"}, inplace=True)
states_in_df_deaths = set(df_deaths_jhu['Province_State'])

# everything in policy df is in deaths df - this sees what in deaths df but not in policy
diff_states = states_in_df_deaths.difference(states_in_policy)
rows_to_remove = df_deaths_jhu[df_deaths_jhu['Province_State'].isin(diff_states)].index
df_deaths_jhu.drop(rows_to_remove, inplace=True)
df_deaths_jhu = df_deaths_jhu.reset_index()

In [16]:
state_population = df_deaths_jhu[['Province_State', 'Population']]

In [17]:
df_deaths_jhu = df_deaths_jhu.drop(['index','UID', 'code3', 'FIPS', 'Lat', 'Long_', 'Population'], axis=1)

In [18]:
date_list = list(df_deaths_jhu.columns)[1:] # index 0 has the state

In [19]:
temp_deaths_df = df_deaths_jhu.copy()
deaths_df = pd.DataFrame(columns=('Date', 'State', 'Deaths Per Capita'))

Note: This cell takes a little while to run approx 48.62 seconds

In [20]:
for index, row in temp_deaths_df.iterrows():
    for date in date_list:
        state = row['Province_State']
        new_row = pd.Series({'Date':date, 'State':state,
                             'Deaths Per Capita':row[date] / 
                             int(state_population[state_population['Province_State'] == state]['Population'])})
        deaths_df = deaths_df.append(new_row, ignore_index= True)

# JHU Data: Covid-19 Cases

In [21]:
df_cases_jhu = org_jhu_cases_df.groupby('Province_State').sum()
df_cases_jhu = df_cases_jhu.reset_index()

In [22]:
df_cases_jhu['Province_State'].replace({"District of Columbia": "Washington DC"}, inplace=True)
states_in_df_cases = set(df_cases_jhu['Province_State'])

rows_to_remove = df_cases_jhu[df_cases_jhu['Province_State'].isin(diff_states)].index
df_cases_jhu.drop(rows_to_remove, inplace=True)
df_cases_jhu = df_cases_jhu.reset_index()

In [23]:
temp_cases_df = df_cases_jhu.copy()

Note: This cell takes a little while to run approx 47.93 seconds

In [24]:
cases_df = pd.DataFrame(columns=('Date', 'State', 'Cases Per Capita'))
for index, row in temp_cases_df.iterrows():
    for date in date_list:
        state = row['Province_State']
        new_row = pd.Series({'Date':date, 'State':state,
                             'Cases Per Capita': row[date] /
                              int(state_population[state_population['Province_State'] == state]['Population'])})
        cases_df = cases_df.append(new_row, ignore_index= True)

In [52]:
# cases_df

In [53]:
# deaths_df

In [54]:
# df_policy

In [55]:
deaths_and_cases_df = pd.merge(deaths_df, cases_df,  how='left', left_on=['Date','State'], 
                            right_on = ['Date','State'])
deaths_and_cases_df['Date'] = pd.to_datetime(deaths_and_cases_df['Date'], format='%m/%d/%y')

In [56]:
# deaths_and_cases_df ends at 10/26 while df_policy ends at 10/27
# Here we are setting restrictingt the dates of df_policy to be stop at the max date of deaths_and_cases_df
df_policy = df_policy[df_policy['Date'] <=  deaths_and_cases_df['Date'].max()]

In [57]:
len(df_policy) == len(deaths_and_cases_df)

True

In [58]:
covid_combined_df = pd.merge(deaths_and_cases_df, df_policy, how='inner',
                               left_on=['Date', 'State'], right_on=['Date', 'RegionName'])

In [59]:
# Checking there is no null values in the new  combined df
temp = covid_combined_df[covid_combined_df.isnull().any(axis=1)]
temp

,Date,State,Deaths Per Capita,Cases Per Capita,RegionName,CountryCode,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay


In [60]:
# Checking to confirm the number of rows in our dataframes are consistent and the new df is the correct len
len(covid_combined_df) == len(df_policy) == len(deaths_and_cases_df)

True